In [1]:
from keras.models import load_model;
from keras.models import Model
# load the image files directly as an array of pixels
from keras.preprocessing.image import load_img  
from keras.applications.vgg16 import VGG16
# pixel data needs to be converted to a NumPy array
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from os import listdir
from pickle import dump

/home/gis-local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# VGG16 is a pretrained model on imagenet dataset for image classification 
# it is pretained model so we don't need to train it again.
# model can be downloaded from https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5
# and places under ~/.keras/models/

model = VGG16();
model.save('../../vgg16.h5');
vggModel = load_model('../../vgg16.h5');

/home/gis-local/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
# will load each photo, prepare it for VGG, and collect the predicted features from the VGG model.
# The image features are a 1-dimensional 4,096 element vector.
vggModel.layers.pop()
vggModel = Model(inputs=vggModel.inputs, outputs=vggModel.layers[-1].output)

# summarize
print(vggModel.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
# we will use VGG model to extract features
# extract features from each photo and save in file 
# directory is path of dataset
# featurePath is where you want to store fetaures.

def extract_features(directory, featurePath):
    features = dict()

    for name in listdir(directory):
        # load an image from file
        filename = directory + '/' + name
        # convert image into 224*224*3
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model  (convert into 1*224, 224)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        feature = vggModel.predict(image, verbose=0)
        # get image id
        image_id = name.split('.')[0]
        # store feature
        features[image_id] = feature
        
    print('Extracted Features: %d' % len(features))
    # save to file
    dump(features, open(featurePath, 'wb'))
    return

In [5]:
# extract features from each photo in the directory
def extract_file_features(filename):
    # load the model
    model = VGG16()
    # re-structure the model
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    # load the photo
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    return feature
 